## Part 1

For this, we are given a few nodes and the connections to other nodes (edges). First consideration was using BFS. However, this causes combinational explosion since there are many paths to explore. Instead, we can use DFS with memoization (caching) to store the number of paths from each node to the end. This way, we only compute the number of paths from each node once. This is a lot less useful than using it with BFS since we want to be able to re-use the computed paths from nodes deeper in the graph. BFS would not allow this since we explore layer by layer.

In [5]:
from functools import lru_cache

# Parse the input into an Adjacency List
def parse_input(raw_input):
    graph = {}
    for line in raw_input.strip().split('\n'):
        if not line: continue
        # "bbb: ddd eee" -> src="bbb", dsts="ddd eee"
        src, dsts_str = line.split(':')
        dsts = dsts_str.strip().split()
        graph[src.strip()] = dsts
    return graph

def solve_reactor(raw_input):
    graph = parse_input(raw_input)

    # Memoization: The "Brain" of the operation.
    # If we ask count_paths('ccc') once, we never solve it again.
    @lru_cache(maxsize=None)
    def count_paths(node):
        # Base Case: We reached the end
        if node == 'out':
            return 1

        # If node has no outgoing connections (dead end), path count is 0
        if node not in graph:
            return 0

        # Recursive Step: Sum of paths from all children
        total_paths = 0
        for neighbor in graph[node]:
            total_paths += count_paths(neighbor)

        return total_paths

    return count_paths('you')

with  open('input.txt') as f:
    print(f"Paths found: {solve_reactor(f.read())}")

Paths found: 523


## Part 2

For this one, we need to ensure that the path triggered actually some nodes that it passed through. So we need to track which nodes have been visited in the current path AND that reach out. When using it in combination with a visited set, we should not store the entire path (this causes massive memory usage), but only the visited nodes that matter. Along this route, only `dac` and `fft` matter.

In [7]:
def solve_reactor(raw_input):
    graph = parse_input(raw_input)

    # Memoization: The "Brain" of the operation.
    # If we ask count_paths('ccc') once, we never solve it again.
    @lru_cache(maxsize=None)
    def count_paths(node, visited = frozenset()):
        # Base Case: We reached the end
        if node == 'out' and {'dac', 'fft'}.issubset(visited):
            return 1

        # If node has no outgoing connections (dead end), path count is 0
        if node not in graph:
            return 0

        # Recursive Step: Sum of paths from all children
        total_paths = 0
        for neighbor in graph[node]:
            new_visited = visited.copy()
            if neighbor in ('dac', 'fft'):
                new_visited = visited | {neighbor}
            total_paths += count_paths(neighbor, new_visited)

        return total_paths

    return count_paths('svr')

with  open('input.txt') as f:
    print(f"Paths found: {solve_reactor(f.read())}")

Paths found: 517315308154944
